# Understanding `.groupby()`

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from env import host, password, user

def get_db_url(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

## Acquire Data

In [2]:
chipotle_sql_query = '''
                     SELECT *
                     FROM orders;
                     '''

In [3]:
orders = pd.read_sql(chipotle_sql_query, get_db_url('chipotle'))
orders.head(10)

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
5,6,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]",$10.98
6,7,3,1,Side of Chips,nan,$1.69
7,8,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour...",$11.75
8,9,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Cheese, Sour Cream, Lettuce]]",$9.25
9,10,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto Beans, Cheese, Sour Cream, Lettuce]]",$9.25


## Prepare Data

In [4]:
# Let's check our data types and do any prep needed to procede with our dataset.

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4622 non-null   int64 
 1   order_id            4622 non-null   int64 
 2   quantity            4622 non-null   int64 
 3   item_name           4622 non-null   object
 4   choice_description  4622 non-null   object
 5   item_price          4622 non-null   object
dtypes: int64(3), object(3)
memory usage: 216.8+ KB


In [5]:
# Ok, now we can do math with our item_price column.

orders['item_price'] = orders.item_price.str.replace('$', '').astype('float')
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  4622 non-null   int64  
 1   order_id            4622 non-null   int64  
 2   quantity            4622 non-null   int64  
 3   item_name           4622 non-null   object 
 4   choice_description  4622 non-null   object 
 5   item_price          4622 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 216.8+ KB


- `item_price` is actually a calcuation of the `item_price` times the `quantity`. I want to rename this column `total` and create a calculated column named `item_price` that is truly the price of each individual item.

In [54]:
# Rename `item_price` to `total`.

orders['total'] = orders['item_price']
orders.head(10)

,id,order_id,quantity,item_name,choice_description,item_price,total
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39,2.39
1,2,1,1,Izze,[Clementine],3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",8.49,8.49
5,6,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]",10.98,10.98
6,7,3,1,Side of Chips,nan,1.69,1.69
7,8,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour...",11.75,11.75
8,9,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Cheese, Sour Cream, Lettuce]]",9.25,9.25
9,10,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto Beans, Cheese, Sour Cream, Lettuce]]",9.25,9.25


In [7]:
# Divide item_price by quantity to get a true item price.

orders['item_price'] = orders.item_price / orders.quantity
orders.head()

,id,order_id,quantity,item_name,choice_description,item_price,total
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39,2.39
1,2,1,1,Izze,[Clementine],3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",8.49,16.98


## Explore Data

In [53]:
# List of unique values in the item_name columns from the orders dataset sorted alphabetically.

item_list = sorted(orders.item_name.unique().tolist())
item_list

['6 Pack Soft Drink',
 'Barbacoa Bowl',
 'Barbacoa Burrito',
 'Barbacoa Crispy Tacos',
 'Barbacoa Salad Bowl',
 'Barbacoa Soft Tacos',
 'Bottled Water',
 'Bowl',
 'Burrito',
 'Canned Soda',
 'Canned Soft Drink',
 'Carnitas Bowl',
 'Carnitas Burrito',
 'Carnitas Crispy Tacos',
 'Carnitas Salad',
 'Carnitas Salad Bowl',
 'Carnitas Soft Tacos',
 'Chicken Bowl',
 'Chicken Burrito',
 'Chicken Crispy Tacos',
 'Chicken Salad',
 'Chicken Salad Bowl',
 'Chicken Soft Tacos',
 'Chips',
 'Chips and Fresh Tomato Salsa',
 'Chips and Guacamole',
 'Chips and Mild Fresh Tomato Salsa',
 'Chips and Roasted Chili Corn Salsa',
 'Chips and Roasted Chili-Corn Salsa',
 'Chips and Tomatillo Green Chili Salsa',
 'Chips and Tomatillo Red Chili Salsa',
 'Chips and Tomatillo-Green Chili Salsa',
 'Chips and Tomatillo-Red Chili Salsa',
 'Crispy Tacos',
 'Izze',
 'Nantucket Nectar',
 'Salad',
 'Side of Chips',
 'Steak Bowl',
 'Steak Burrito',
 'Steak Crispy Tacos',
 'Steak Salad',
 'Steak Salad Bowl',
 'Steak Soft Ta

In [52]:
# We can see that this matches the unique items in our groupby object.

groupby_items = [item for item, subset in orders.groupby('item_name')]
groupby_items

['6 Pack Soft Drink',
 'Barbacoa Bowl',
 'Barbacoa Burrito',
 'Barbacoa Crispy Tacos',
 'Barbacoa Salad Bowl',
 'Barbacoa Soft Tacos',
 'Bottled Water',
 'Bowl',
 'Burrito',
 'Canned Soda',
 'Canned Soft Drink',
 'Carnitas Bowl',
 'Carnitas Burrito',
 'Carnitas Crispy Tacos',
 'Carnitas Salad',
 'Carnitas Salad Bowl',
 'Carnitas Soft Tacos',
 'Chicken Bowl',
 'Chicken Burrito',
 'Chicken Crispy Tacos',
 'Chicken Salad',
 'Chicken Salad Bowl',
 'Chicken Soft Tacos',
 'Chips',
 'Chips and Fresh Tomato Salsa',
 'Chips and Guacamole',
 'Chips and Mild Fresh Tomato Salsa',
 'Chips and Roasted Chili Corn Salsa',
 'Chips and Roasted Chili-Corn Salsa',
 'Chips and Tomatillo Green Chili Salsa',
 'Chips and Tomatillo Red Chili Salsa',
 'Chips and Tomatillo-Green Chili Salsa',
 'Chips and Tomatillo-Red Chili Salsa',
 'Crispy Tacos',
 'Izze',
 'Nantucket Nectar',
 'Salad',
 'Side of Chips',
 'Steak Bowl',
 'Steak Burrito',
 'Steak Crispy Tacos',
 'Steak Salad',
 'Steak Salad Bowl',
 'Steak Soft Ta

In [63]:
# There are 726 observations in the Chicken Bowl subset when we groupby item_name.

chicken_bowl_subset = [subset[['item_name', 'item_price']] for item, subset in orders.groupby('item_name') if item == 'Chicken Bowl']
chicken_bowl_subset

[         item_name  item_price
 4     Chicken Bowl        8.49
 5     Chicken Bowl       10.98
 13    Chicken Bowl       11.25
 19    Chicken Bowl        8.75
 26    Chicken Bowl        8.49
 ...            ...         ...
 4590  Chicken Bowl       11.25
 4591  Chicken Bowl        8.75
 4595  Chicken Bowl        8.75
 4599  Chicken Bowl        8.75
 4604  Chicken Bowl        8.75
 
 [726 rows x 2 columns]]

In [64]:
# There are 2 observations in the Crispy Tacos subset when we groupby item_name.

crispy_tacos_subset = [subset[['item_name', 'item_price']] for item, subset in orders.groupby('item_name') if item == 'Crispy Tacos']
crispy_tacos_subset

[        item_name  item_price
 520  Crispy Tacos         7.4
 521  Crispy Tacos         7.4]

In [43]:
# There are 7 observations in the Veggie Soft Tacos subset when we groupby item_name.

veggie_soft_tacos_subset = [subset[['item_name', 'item_price']] for item, subset in orders.groupby('item_name') if item == 'Veggie Soft Tacos']
veggie_soft_tacos_subset

[              item_name  item_price
 738   Veggie Soft Tacos       11.25
 781   Veggie Soft Tacos        8.75
 1395  Veggie Soft Tacos        8.49
 1699  Veggie Soft Tacos       11.25
 2384  Veggie Soft Tacos        8.75
 2851  Veggie Soft Tacos        8.49
 3889  Veggie Soft Tacos        8.49]

**If we want to reshape our data such that each observation or row is a unique item, we have to decide what to do with all of the information in our other column(s), in this case, `item_price`.**

___

### Use `.mean()` to summarize your data.

In [90]:
orders_item_price_means = orders.groupby('item_name').agg(item_price_mean = ('item_price', 'mean')).round(2)
orders_item_price_means.head()

,item_price_mean
item_name,
6 Pack Soft Drink,6.49
Barbacoa Bowl,10.19
Barbacoa Burrito,9.83
Barbacoa Crispy Tacos,10.09
Barbacoa Salad Bowl,10.64


In [37]:
# Focus on a subset, Chicken Bowl, Crispy Tacos, Veggie Soft Tacos, for demo purposes.

orders_item_price_means.loc[['Chicken Bowl', 'Crispy Tacos', 'Veggie Soft Tacos']]

,item_price_mean
item_name,
Chicken Bowl,9.66
Crispy Tacos,7.40
Veggie Soft Tacos,9.35


___

#### `.mean()` Breakdown

- Here we can see that all 726 Chicken Bowl observations have been collapsed into a single observation by summarizing all of the values for `item_price` using their mean value.

In [38]:
# This subset of Chicken Bowl observations...

chicken_bowl_subset

[         item_name  item_price
 4     Chicken Bowl        8.49
 5     Chicken Bowl       10.98
 13    Chicken Bowl       11.25
 19    Chicken Bowl        8.75
 26    Chicken Bowl        8.49
 ...            ...         ...
 4590  Chicken Bowl       11.25
 4591  Chicken Bowl        8.75
 4595  Chicken Bowl        8.75
 4599  Chicken Bowl        8.75
 4604  Chicken Bowl        8.75
 
 [726 rows x 2 columns]]

In [39]:
# ...is now one summary observation.

orders_item_price_means.loc[['Chicken Bowl']]

,item_price_mean
item_name,
Chicken Bowl,9.66


- Here we can see that the 2 Crispy Taco observations have been collapsed into a single observation by summarizing all of the values for `item_price` using their mean value.

In [40]:
# This subset of Crispy Tacos observations...

crispy_tacos_subset

[        item_name  item_price
 520  Crispy Tacos         7.4
 521  Crispy Tacos         7.4]

In [41]:
# ...is now one summary observation.

orders_item_price_means.loc[['Crispy Tacos']]

,item_price_mean
item_name,
Crispy Tacos,7.4


- Here we can see that all 7 Veggie Soft Taco observations have been collapsed into a single observation by summarizing all of the values for `item_price` using their mean value.

In [44]:
# This subset of Veggie Soft Tacos observations...

veggie_soft_tacos_subset

[              item_name  item_price
 738   Veggie Soft Tacos       11.25
 781   Veggie Soft Tacos        8.75
 1395  Veggie Soft Tacos        8.49
 1699  Veggie Soft Tacos       11.25
 2384  Veggie Soft Tacos        8.75
 2851  Veggie Soft Tacos        8.49
 3889  Veggie Soft Tacos        8.49]

In [45]:
# ...is now one summary observation.

orders_item_price_means.loc[['Veggie Soft Tacos']]

,item_price_mean
item_name,
Veggie Soft Tacos,9.35


___

### Use `.median()` to summarize your data.

In [88]:
orders_item_price_median = orders.groupby('item_name').agg(item_price_median = ('item_price', 'median'))
orders_item_price_median.reset_index().head()

,item_name,item_price_median
0,6 Pack Soft Drink,6.49
1,Barbacoa Bowl,9.25
2,Barbacoa Burrito,9.25
3,Barbacoa Crispy Tacos,9.25
4,Barbacoa Salad Bowl,10.64


In [89]:
# Focus on a subset, Chicken Bowl, Crispy Tacos, Veggie Soft Tacos, for demo purposes.

orders_item_price_median.loc[['Chicken Bowl', 'Crispy Tacos', 'Veggie Soft Tacos']]

,item_price_median
item_name,
Chicken Bowl,8.75
Crispy Tacos,7.40
Veggie Soft Tacos,8.75


___

### Use `.count()` to summarize your data.

In [84]:
orders_item_count = orders.groupby('item_name').agg(item_count = ('item_price', 'count'))
orders_item_count.reset_index().head()

,item_name,item_count
0,6 Pack Soft Drink,54
1,Barbacoa Bowl,66
2,Barbacoa Burrito,91
3,Barbacoa Crispy Tacos,11
4,Barbacoa Salad Bowl,10


In [85]:
# Focus on a subset, Chicken Bowl, Crispy Tacos, Veggie Soft Tacos, for demo purposes.

orders_item_count.loc[['Chicken Bowl', 'Crispy Tacos', 'Veggie Soft Tacos']]

,item_count
item_name,
Chicken Bowl,726
Crispy Tacos,2
Veggie Soft Tacos,7


___

### Use `.sum()` to summarize your data.

In [86]:
orders_item_price_sum = orders.groupby('item_name').agg(item_price_sum = ('item_price', 'sum'))
orders_item_price_sum.reset_index().head()

,item_name,item_price_sum
0,6 Pack Soft Drink,350.46
1,Barbacoa Bowl,672.36
2,Barbacoa Burrito,894.75
3,Barbacoa Crispy Tacos,110.96
4,Barbacoa Salad Bowl,106.40


In [87]:
# Focus on a subset, Chicken Bowl, Crispy Tacos, Veggie Soft Tacos, for demo purposes.

orders_item_price_sum.loc[['Chicken Bowl', 'Crispy Tacos', 'Veggie Soft Tacos']]

,item_price_sum
item_name,
Chicken Bowl,7011.51
Crispy Tacos,14.80
Veggie Soft Tacos,65.47
